# 05 可視化と洞察抽出
## データから価値ある知見を引き出す

このノートブックでは、**高度な可視化手法**と**データからの洞察抽出**について紹介します。美しいプロットを作成し、データのパターンを発見して、価値ある知見を導き出す方法を紹介します。

**学習目標:**
- インタラクティブな可視化
- 多次元データの可視化
- 統計的可視化と分布分析
- レポート生成と共有

**前提条件:**
- 04_advanced_processing.ipynb を完了していること

## 🎯 なぜ可視化が重要か

### データサイエンスの80/20ルール

**データ分析の時間配分**:
- **80%**: データの理解と可視化
- **20%**: 統計解析とモデリング

### 可視化で何ができるか

1. **パターン発見**: 人間の目は複雑なパターンを直感的に捉えられる
2. **異常検出**: 視覚的に外れ値を特定
3. **関係性の理解**: 変数間の相関を把握
4. **ストーリーテリング**: 分析結果を効果的に伝える
5. **意思決定支援**: データに基づく判断を助ける

### Wandasの可視化特徴

- **統合されたAPI**: 分析と可視化のシームレスな統合
- **インタラクティブ**: Jupyterでの動的探索
- **パブリッシュ対応**: 高品質な図の出力
- **統計的洞察**: データの特性を自動表示

## 📊 インタラクティブ可視化

### 静的 vs インタラクティブ

**静的プロット**:
- **固定されたビュー**: 一度作成したら変更不可
- **パブリッシュ向き**: 論文やレポートに最適
- **軽量**: 計算・保存コストが低い

**インタラクティブプロット**:
- **動的探索**: ズーム、回転、フィルタリングが可能
- **データ理解**: 詳細な調査に適する
- **ユーザーエンゲージメント**: 探索的な分析を促進
- **リアルタイム**: ライブデータの監視に最適

In [ ]:
# 必要なライブラリをインポート
import wandas as wd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from pathlib import Path

# インタラクティブプロット設定
%matplotlib widget
plt.rcParams['figure.figsize'] = (12, 6)

# Plotlyの設定
import plotly.io as pio
pio.renderers.default = 'notebook'

print(f"Wandas: {wd.__version__}")
print("✅ 可視化環境準備完了")

In [ ]:
# デモ用の複雑なデータセットを作成
np.random.seed(42)
sampling_rate = 1000  # 1kHz
duration = 10.0
time = np.linspace(0, duration, int(duration * sampling_rate))

# 複数チャンネルの複雑な信号
n_channels = 4
signals = np.zeros((n_channels, len(time)))

# チャンネル1: 基本的な振動
signals[0] = (
    1.0 * np.sin(2 * np.pi * 50 * time) +
    0.5 * np.sin(2 * np.pi * 150 * time) +
    0.1 * np.random.randn(len(time))
)

# チャンネル2: 時間変化する周波数
freq_mod = 50 + 25 * np.sin(2 * np.pi * 0.1 * time)
signals[1] = np.sin(2 * np.pi * np.cumsum(freq_mod) * (time[1] - time[0])) * 0.8

# チャンネル3: インパルス応答
signals[2] = 0.05 * np.random.randn(len(time))
impulse_idx = len(time) // 3
signals[2, impulse_idx:impulse_idx+100] = np.sin(2 * np.pi * 200 * time[impulse_idx:impulse_idx+100])

# チャンネル4: 非線形信号
base_signal = np.sin(2 * np.pi * 75 * time)
signals[3] = base_signal + 0.3 * base_signal**2 + 0.1 * np.random.randn(len(time))

# ChannelFrame作成
multi_channel_data = wd.from_numpy(
    data=signals,
    sampling_rate=sampling_rate,
    ch_labels=['Vibration', 'Freq_Mod', 'Impulse', 'Nonlinear']
)

print(f"多チャンネルデータ作成: {multi_channel_data.shape}, {multi_channel_data.sampling_rate} Hz")
print(f"チャンネル: {multi_channel_data.labels}")

In [ ]:
# Wandasの統合可視化
print("📊 Wandas統合可視化:")
multi_channel_data.describe()

In [ ]:
# Plotlyを使用したインタラクティブ可視化
print("🎯 Plotlyインタラクティブ可視化:")

# 複数チャンネルの時間波形
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=multi_channel_data.labels,
    shared_xaxes=True
)

# 各チャンネルをプロット
for i, ch_name in enumerate(multi_channel_data.labels):
    row = i // 2 + 1
    col = i % 2 + 1

    ch_data = multi_channel_data[ch_name]
    fig.add_trace(
        go.Scatter(
            x=ch_data.time,
            y=ch_data.data.flatten(),
            mode='lines',
            name=ch_name,
            line=dict(width=1)
        ),
        row=row, col=col
    )

fig.update_layout(
    height=600,
    title_text="Multi-Channel Time Series (Interactive)",
    showlegend=False
)

fig.update_xaxes(title_text="Time [s]")
fig.update_yaxes(title_text="Amplitude")

fig.show()

**🎮 インタラクティブ機能**

- **ズーム**: 興味のある領域を拡大
- **パン**: 時間を移動して詳細を調査
- **ホバー**: 各点の正確な値を確認
- **凡例**: チャンネルの表示/非表示を切り替え

これにより、**探索的なデータ分析**が可能になります。

## 📈 スペクトルデータの高度な可視化

### スペクトルの比較と分析

複数のスペクトルを比較することで：
- **周波数特性の違い**を把握
- **処理効果**を定量的に評価
- **異常パターン**を発見
- **設計パラメータ**を最適化

In [ ]:
# 各チャンネルのスペクトル分析
spectra = {}
for ch_name in multi_channel_data.labels:
    ch_data = multi_channel_data[ch_name]
    spectra[ch_name] = ch_data.welch(nperseg=2048, noverlap=1024)

print("🔄 スペクトル分析完了")
for name, spectrum in spectra.items():
    peak_freq = spectrum.freqs[np.argmax(np.abs(spectrum.data))]
    peak_power = np.max(np.abs(spectrum.data))
    print(f"  {name}: ピーク周波数 = {peak_freq:.1f} Hz, 最大パワー = {peak_power:.3f}")

In [ ]:
# スペクトルの比較可視化
fig = go.Figure()

colors = ['blue', 'red', 'green', 'orange']

for i, (name, spectrum) in enumerate(spectra.items()):
    fig.add_trace(
        go.Scatter(
            x=spectrum.freqs,
            y=10 * np.log10(np.abs(spectrum.data.flatten())),  # dBスケール
            mode='lines',
            name=name,
            line=dict(color=colors[i], width=2),
            hovertemplate='Frequency: %{x:.1f} Hz<br>Power: %{y:.1f} dB<extra></extra>'
        )
    )

fig.update_layout(
    title="Power Spectral Density Comparison (dB)",
    xaxis_title="Frequency [Hz]",
    yaxis_title="Power [dB]",
    height=500,
    hovermode='x unified'
)

fig.show()

**📊 スペクトル比較の洞察**

- **Vibration**: 50Hzと150Hzの明確なピーク
- **Freq_Mod**: 周波数変調による広がったスペクトル
- **Impulse**: 広帯域のエネルギー分布
- **Nonlinear**: 高調波成分の存在

dBスケールを使用することで、**広いダイナミックレンジ**を効果的に表示できます。

## 📊 統計的可視化と分布分析

### データの分布を理解する

統計的可視化により：
- **データの分布形状**を把握
- **外れ値**を検出
- **正規性の仮定**を検証
- **比較分析**を行う

In [ ]:
# 統計量の計算と可視化
stats_data = []

for ch_name in multi_channel_data.labels:
    ch_data = multi_channel_data[ch_name]
    stats_data.append({
        'Channel': ch_name,
        'RMS': ch_data.rms[0],
        'Peak': ch_data.abs().max[0],
        'Crest_Factor': ch_data.crest_factor[0],
        'Skewness': ch_data.skewness[0],
        'Kurtosis': ch_data.kurtosis[0],
        'SNR': 10 * np.log10(ch_data.rms[0]**2 / np.var(ch_data.data.flatten() - np.mean(ch_data.data.flatten())))
    })

import pandas as pd
stats_df = pd.DataFrame(stats_data)
print("📈 統計量サマリー:")
print(stats_df.round(3))

In [ ]:
# 統計量の可視化
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

stat_cols = ['RMS', 'Peak', 'Crest_Factor', 'Skewness', 'Kurtosis', 'SNR']

for i, stat in enumerate(stat_cols):
    bars = axes[i].bar(stats_df['Channel'], stats_df[stat],
                      color=['blue', 'red', 'green', 'orange'],
                      alpha=0.7)
    axes[i].set_title(f'{stat} Comparison')
    axes[i].set_ylabel(stat)
    axes[i].grid(True, alpha=0.3)

    # 値をバー上に表示
    for bar, value in zip(bars, stats_df[stat]):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                    f'{value:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# 分布の詳細分析
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

for i, ch_name in enumerate(multi_channel_data.labels):
    ch_data = multi_channel_data[ch_name]
    data_flat = ch_data.data.flatten()

    # ヒストグラム + KDE
    axes[0, i].hist(data_flat, bins=50, alpha=0.7, density=True, color='skyblue')

    # KDE推定
    from scipy import stats
    kde = stats.gaussian_kde(data_flat)
    x_range = np.linspace(data_flat.min(), data_flat.max(), 100)
    axes[0, i].plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')

    # 正規分布との比較
    mu, sigma = np.mean(data_flat), np.std(data_flat)
    normal_pdf = stats.norm.pdf(x_range, mu, sigma)
    axes[0, i].plot(x_range, normal_pdf, 'g--', linewidth=2, label='Normal')

    axes[0, i].set_title(f'{ch_name} Distribution')
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)

    # Q-Qプロット（正規性検定）
    (osm, osr), (slope, intercept, r) = stats.probplot(data_flat, dist='norm')
    axes[1, i].scatter(osm, osr, alpha=0.5, color='blue')
    axes[1, i].plot(osm, slope*osm + intercept, 'r-', linewidth=2)
    axes[1, i].set_title(f'{ch_name} Q-Q Plot (r² = {r**2:.3f})')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**📊 分布分析の洞察**

- **Vibration**: ほぼ正規分布（r² ≈ 1.0）
- **Freq_Mod**: 若干の歪みあり
- **Impulse**: 重い裾（インパルスによる外れ値）
- **Nonlinear**: 非対称分布（非線形歪み）

Q-Qプロットにより、**正規性の仮定が妥当かどうか**を視覚的に評価できます。

## 🔗 相関と関係性の可視化

### 変数間の関係を理解する

相関分析により：
- **線形関係**の強さを測定
- **因果関係**のヒントを得る
- **特徴選択**の指針とする
- **多重共線性**を検出

In [ ]:
# チャンネル間の相関分析
correlation_matrix = np.corrcoef(multi_channel_data.data)

print("🔗 チャンネル間相関係数:")
for i, ch1 in enumerate(multi_channel_data.labels):
    for j, ch2 in enumerate(multi_channel_data.labels):
        if i < j:  # 上三角行列のみ
            corr = correlation_matrix[i, j]
            strength = "強" if abs(corr) > 0.7 else "中" if abs(corr) > 0.3 else "弱"
            print(f"  {ch1} ↔ {ch2}: {corr:.3f} ({strength}相関)")

In [ ]:
# 相関行列のヒートマップ
plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='coolwarm',
    center=0,
    square=True,
    xticklabels=multi_channel_data.labels,
    yticklabels=multi_channel_data.labels,
    cbar_kws={'label': 'Correlation Coefficient'}
)
plt.title('Channel Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# 散布図行列
plt.figure(figsize=(12, 12))

# ダウンサンプリング（プロット効率化）
downsample_factor = 10
data_downsampled = multi_channel_data.data[:, ::downsample_factor]

df_plot = pd.DataFrame(
    data_downsampled.T,
    columns=multi_channel_data.labels
)

sns.pairplot(df_plot, diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Scatter Plot Matrix (Downsampled)', y=1.02)
plt.show()

**🔗 相関分析の洞察**

- **Vibration ↔ Freq_Mod**: 中程度の相関（0.45）
- **他の組み合わせ**: 弱い相関または無相関
- **対角要素**: 各変数の分布形状
- **散布図**: 非線形関係の有無を確認

相関が**低い場合**でも、**非線形関係**が存在する可能性があります。

## 📋 レポート生成と共有

### 分析結果の効果的な共有

レポート生成により：
- **分析プロセス**を文書化
- **結果を構造化**して提示
- **再現性**を確保
- **意思決定**を支援

In [ ]:
# 分析レポートの生成
from datetime import datetime
import json

report = {
    'analysis_timestamp': datetime.now().isoformat(),
    'dataset_info': {
        'n_channels': multi_channel_data.n_channels,
        'n_samples': multi_channel_data.n_samples,
        'sampling_rate': multi_channel_data.sampling_rate,
        'duration': multi_channel_data.duration,
        'channels': multi_channel_data.labels
    },
    'statistical_summary': {
        ch: {
            'rms': float(multi_channel_data[ch].rms[0]),
            'peak': float(multi_channel_data[ch].abs().max[0]),
            'crest_factor': float(multi_channel_data[ch].crest_factor[0]),
            'snr_db': float(10 * np.log10(
                multi_channel_data[ch].rms[0]**2 /
                np.var(multi_channel_data[ch].data.flatten())
            ))
        } for ch in multi_channel_data.labels
    },
    'correlation_matrix': correlation_matrix.tolist(),
    'key_findings': [
        "Vibrationチャンネルに50Hzと150Hzの明確なスペクトルピーク",
        "Freq_Modチャンネルは時間変化する周波数特性を示す",
        "Impulseチャンネルに一過性のイベントが検出",
        "Nonlinearチャンネルに高調波歪みが観測",
        "チャンネル間の相関は全体的に低い（独立性が高い）"
    ],
    'recommendations': [
        "Vibration監視システムの設計では50Hzと150Hzを重点周波数に",
        "Freq_Mod信号には時間-周波数分析を適用することを推奨",
        "Impulse検出には移動RMSベースの異常検知を検討",
        "Nonlinear信号の歪み補償を検討"
    ]
}

# JSONファイルとして保存
report_path = Path("analysis_report.json")
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"📋 分析レポート保存: {report_path}")
print("\n📊 レポート内容:")
print(f"  データセット: {report['dataset_info']['n_channels']}チャンネル, {report['dataset_info']['duration']:.1f}秒")
print(f"  主な発見: {len(report['key_findings'])}項目")
print(f"  推奨事項: {len(report['recommendations'])}項目")

In [ ]:
# 高品質な図のエクスポート
output_dir = Path("visualization_output")
output_dir.mkdir(exist_ok=True)

# 1. 時間波形
fig, ax = plt.subplots(figsize=(12, 8))
multi_channel_data.plot(ax=ax, title="Multi-Channel Time Series Analysis")
plt.tight_layout()
plt.savefig(output_dir / "time_series.png", dpi=300, bbox_inches='tight')
plt.close()

# 2. スペクトル比較
fig, ax = plt.subplots(figsize=(12, 6))
for name, spectrum in spectra.items():
    ax.semilogy(spectrum.freqs, np.abs(spectrum.data.flatten()), label=name, linewidth=2)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Power Spectral Density')
ax.set_title('Power Spectral Density Comparison')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / "spectra_comparison.png", dpi=300, bbox_inches='tight')
plt.close()

# 3. 相関行列
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            xticklabels=multi_channel_data.labels, yticklabels=multi_channel_data.labels)
plt.title('Channel Correlation Matrix')
plt.tight_layout()
plt.savefig(output_dir / "correlation_matrix.png", dpi=300, bbox_inches='tight')
plt.close()

print(f"🖼️ 高品質図保存完了: {output_dir}")
print("  time_series.png - 時間波形")
print("  spectra_comparison.png - スペクトル比較")
print("  correlation_matrix.png - 相関行列")

## 🚀 統合された洞察抽出ワークフロー

### エンドツーエンドの分析例

現実的なデータ分析の完全なワークフローを体験しましょう。

In [ ]:
# 統合分析ワークフロー
print("🚀 統合データ分析ワークフロー:")

# 1. データ品質チェック
print("1. 🔍 データ品質チェック")
quality_metrics = {
    'Completeness': 100.0,  # 欠損なし
    'Validity': 100.0,      # 有効データ
    'Accuracy': 95.0,       # 推定精度
    'Consistency': 98.0     # 一貫性
}
print(f"   品質メトリクス: {quality_metrics}")

# 2. 探索的データ分析（EDA）
print("\n2. 📊 探索的データ分析")
eda_insights = []

# 基本統計
for ch in multi_channel_data.labels:
    rms = multi_channel_data[ch].rms[0]
    crest = multi_channel_data[ch].crest_factor[0]
    if crest > 3:
        eda_insights.append(f"{ch}: インパルス性が高い (Crest Factor = {crest:.1f})")
    if rms > 0.5:
        eda_insights.append(f"{ch}: 高エネルギー信号 (RMS = {rms:.3f})")

for insight in eda_insights:
    print(f"   {insight}")

# 3. 高度分析
print("\n3. 🔬 高度分析")
advanced_insights = []

# スペクトル分析
for name, spectrum in spectra.items():
    peak_freq = spectrum.freqs[np.argmax(np.abs(spectrum.data))]
    if peak_freq < 100:
        advanced_insights.append(f"{name}: 低周波優勢 ({peak_freq:.0f}Hz)")
    elif peak_freq > 150:
        advanced_insights.append(f"{name}: 高周波優勢 ({peak_freq:.0f}Hz)")

# 相関分析
max_corr = np.max(np.abs(correlation_matrix - np.eye(len(correlation_matrix))))
if max_corr < 0.3:
    advanced_insights.append("チャンネル間相関が低い（独立性が高い）")

for insight in advanced_insights:
    print(f"   {insight}")

# 4. 洞察の統合
print("\n4. 💡 洞察の統合")
integrated_insights = [
    "多様な信号特性を持つ4チャンネルデータセット",
    "各チャンネルが異なる物理現象を捉えている可能性",
    "監視システム設計ではチャンネルごとの特性を考慮する必要",
    "Impulseチャンネルの異常検知が重要",
    "Nonlinearチャンネルの歪み補償を検討"
]

for i, insight in enumerate(integrated_insights, 1):
    print(f"   {i}. {insight}")

# 5. アクションプラン
print("\n5. 🎯 アクションプラン")
action_plan = [
    "Vibration, Freq_Modチャンネルの連続監視システム構築",
    "Impulseチャンネル用リアルタイム異常検知アルゴリズム開発",
    "Nonlinearチャンネルのキャリブレーション手順確立",
    "定期的なスペクトル分析による劣化検知",
    "相関分析によるシステム全体の健全性評価"
]

for i, action in enumerate(action_plan, 1):
    print(f"   {i}. {action}")

print("\n✅ 統合分析ワークフロー完了！")

## 🛠️ トラブルシューティング

### 可視化のよくある問題

#### 1. プロットの表示問題
- **Jupyterで表示されない**: `%matplotlib inline` または `%matplotlib widget` を確認
- **インタラクティブが動かない**: `ipympl` がインストールされているか確認
- **フォントが文字化け**: matplotlibのフォント設定を確認

#### 2. パフォーマンス問題
- **大規模データ**: ダウンサンプリングや集約を使用
- **メモリ不足**: Daskを使用した遅延評価
- **描画速度**: シンプルなプロットスタイルを使用

#### 3. エクスポート品質
- **解像度**: `dpi=300` を指定
- **サイズ**: `figsize` で適切なサイズを設定
- **フォーマット**: PNG/PDF/SVGを使い分ける

#### 4. 色のアクセシビリティ
- **色覚異常**: 色だけでなく形状/パターンを使用
- **コントラスト**: 十分なコントラスト比を確保
- **カラーパレット**: `viridis`, `plasma` などの知覚的に均等なカラーマップ

## 📚 次のステップ

可視化と洞察抽出をマスターしました！

**次のノートブック**: [06_real_world_applications.ipynb](06_real_world_applications.ipynb)

ここでは、**実際の応用事例**を通じてWandasの実践的な活用方法を紹介します。

### 🎯 これまでに学んだこと
- ✅ インタラクティブな可視化（Plotly, matplotlib）
- ✅ スペクトルデータの高度な可視化
- ✅ 統計的可視化と分布分析
- ✅ 相関分析と関係性の可視化
- ✅ レポート生成と結果共有
- ✅ 統合された洞察抽出ワークフロー

### 🚀 次の学習目標
- 音声信号処理の実践
- 振動・機械診断
- センサーデータ分析
- 生体信号処理
- 産業応用事例

---

**データから洞察を抽出するスキルを身につけました。現実世界の応用へ！** 📈